In [332]:
import cv2
import numpy as np
import os
import pickle

In [333]:
def unpickle(file):
    
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [334]:
d1 = unpickle('cifar-10-batches-py\data_batch_1')
d1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [ ]:
image_names = []
images = []
labels = []

In [335]:
def fun(x):
    for i in x[b'data']:
        i = np.reshape(i, (32,32,3))
        images.append(i)
    for i in x[b'labels']:
        labels.append(i)
    for i in x[b'filenames']:
        image_names.append(i)
    return

In [336]:
d1 = unpickle('cifar-10-batches-py\data_batch_1')
fun(d1) 
d2 = unpickle('cifar-10-batches-py\data_batch_2') 
fun(d2)
d3 = unpickle('cifar-10-batches-py\data_batch_3') 
fun(d3)
d4 = unpickle('cifar-10-batches-py\data_batch_4') 
fun(d4)
d5 = unpickle('cifar-10-batches-py\data_batch_5') 
fun(d5)

In [337]:
len(image_names)

50000

In [338]:
len(images)

50000

In [339]:
len(labels)

50000

In [340]:
sift = cv2.SIFT_create()
imgwise_des_list = []
des_list = [] 

In [341]:
skip_index = []
for t in range(len(images)):
    kpts, des = sift.detectAndCompute(images[t], None)
    if des is not None:
        imgwise_des_list.append((image_names[t], des))
        for i in des:
            des_list.append(i)
    else:
        skip_index.append(t)
        
des_list = np.array(des_list)   

In [342]:
len(skip_index)

233

In [343]:
len(imgwise_des_list)

49767

In [344]:
imgwise_des_list

[(b'leptodactylus_pentadactylus_s_000004.png',
  array([[  1.,   0.,   0., ...,   7.,   2.,  13.],
         [ 14.,   0.,   0., ...,  38.,  14.,  17.],
         [ 23.,  13.,   1., ...,  18.,   0.,   1.],
         ...,
         [  0.,   1.,   1., ...,   6.,  11.,  16.],
         [  0.,   0.,   0., ...,   1.,   0.,   4.],
         [125.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)),
 (b'camion_s_000148.png',
  array([[ 0.,  0.,  1., ..., 12.,  0.,  6.],
         [ 0.,  0.,  0., ..., 22.,  9., 62.],
         [ 2.,  2.,  2., ..., 14., 24., 20.],
         ...,
         [ 6.,  0.,  0., ...,  0.,  0.,  0.],
         [26.,  2.,  0., ...,  0.,  0.,  0.],
         [18.,  5.,  2., ...,  0.,  0.,  0.]], dtype=float32)),
 (b'tipper_truck_s_001250.png',
  array([[ 0.,  0.,  0., ..., 33.,  9.,  7.],
         [29.,  2.,  1., ...,  4.,  0.,  0.],
         [35.,  5.,  0., ...,  4.,  1., 44.],
         ...,
         [ 5.,  1.,  0., ...,  1.,  1., 24.],
         [ 2.,  0.,  0., ..., 13.,  3., 33.],

In [345]:
from scipy.cluster.vq import kmeans, vq
k = 200  
voc, variance = kmeans(des_list, k, 1)

In [346]:
histograms = np.zeros((len(imgwise_des_list), k), "float32")
for i in range(len(imgwise_des_list)):
    words, distance = vq(imgwise_des_list[i][1],voc)
    for w in words:
        histograms[i][w] += 1

In [347]:
from sklearn.preprocessing import StandardScaler
stdSlr = StandardScaler().fit(histograms)
histograms = stdSlr.transform(histograms)

In [348]:
len(labels)

50000

In [349]:
for i in sorted(skip_index, reverse=True):
    del labels[i]
len(labels)

49767

In [350]:
#from xgboost import XGBClassifier
#xgbr = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
#xgbr = XGBClassifier(tree_method='gpu_hist',gpu_id=0,  min_child_weight = 5,
 #max_depth = 130,
 #learning_rate = 0.008,
 #gamma = 0.3,
 #colsample_bytree = 0.7)
#xgbr.fit(histograms,y_train)

In [351]:
#from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression(random_state=0).fit(histograms,y_train)

In [352]:
#from sklearn.svm import LinearSVC
#y_train = np.asarray(labels, dtype=np.float32)
#clf = LinearSVC() 
#clf.fit(histograms,y_train)

In [353]:
from sklearn.svm import SVC
y_train = np.asarray(labels, dtype=np.float32)
clf = SVC(max_iter=70000,C=1.80)
clf.fit(histograms,y_train)


SVC(C=1.8, max_iter=70000)

Testing

In [354]:
d1 = unpickle('cifar-10-batches-py\data_batch_1')
d1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [355]:
image_names = []
images = []
labels = []
def fun(x):
    for i in x[b'data']:
        i = np.reshape(i, (32,32,3))
        images.append(i)
    for i in x[b'labels']:
        labels.append(i)
    for i in x[b'filenames']:
        image_names.append(i)
    return

fun(d1) 

In [356]:
len(labels)

10000

In [357]:
sift = cv2.SIFT_create()
imgwise_des_list = []
des_list = [] 

In [358]:
skip_index = []
for t in range(len(images)):
    kpts, des = sift.detectAndCompute(images[t], None)
    if des is not None:
        imgwise_des_list.append((image_names[t], des))
        for i in des:
            des_list.append(i)
    else:
        skip_index.append(t)
        
des_list = np.array(des_list)   
len(skip_index)

46

In [359]:
histograms = np.zeros((len(imgwise_des_list), k), "float32")
for i in range(len(imgwise_des_list)):
    words, distance = vq(imgwise_des_list[i][1],voc)
    for w in words:
        histograms[i][w] += 1

In [360]:
histograms.shape

(9954, 200)

In [361]:
stdSlr = StandardScaler().fit(histograms)
histograms = stdSlr.transform(histograms)

for i in sorted(skip_index, reverse=True):
    del labels[i]
len(labels)

9954

In [362]:
y_test = np.asarray(labels, dtype=np.float32)
predictions=clf.predict(histograms)
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

print(get_accuracy(predictions,y_test))

[6. 9. 9. ... 1. 8. 5.] [6. 9. 9. ... 1. 1. 5.]
0.7029334940727345
